In [1]:
import pandas as pd
import numpy as np
import json
import gzip
import os
import os.path
from os import listdir
import multiprocessing
from tqdm import tqdm
import mmap
from multiprocessing import Pool
import seaborn as sns
import matplotlib.pyplot as plt
import py_entitymatching as em
import py_stringmatching as sm
import scipy
from scipy.io import arff
from sklearn import preprocessing
#from sklearn.model_selection import train_test_split
from skmultilearn.model_selection import iterative_train_test_split
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
import progressbar
import itertools

In [2]:
product_path = '../../../../src/data/product'
cluster_path = os.path.join(product_path, 'lspc2020_to_tablecorpus/Cleaned')
cleaned_top100_path = os.path.join(product_path, 'product_top100/cleaned')
top100_path = os.path.join(product_path, 'product_top100')
cleaned_min3_path = os.path.join(product_path, 'product_minimum3/cleaned')
min3_path = os.path.join(product_path, 'product_minimum3')
results_path = os.path.join(product_path, 'product_results/after_preprocessing')
train_test_input_filtered_path = os.path.join(product_path, 'train_test_split/input_filtered_tables')
train_test_all_filtered_path = os.path.join(train_test_input_filtered_path, 'all')
train_test_input_unfiltered_path = os.path.join(product_path, 'train_test_split/input_unfiltered_tables')
train_test_output_path = os.path.join(product_path, 'train_test_split/output_unfiltered_tables')

# Generate filtered tables

## testing

In [10]:
'''
entities = ['Bikes', 'Cars', 'Clothes', 'Drugstore', 'Electronics', 'Technology', 'Tools', 'Random']
count = 0
for entity in entities:
    df_category1 = pd.read_csv(os.path.join(mapping_corpus_path_2,
                                            '{}_clusters_all_8_tables_post_processed_lower_threshold.csv'.format(
                                                entity)))
    df_category1['category'] = entity.lower()
    if count == 0:
        df_concat = df_category1
    if count > 0:
        df_concat = pd.concat([df_category1, df_category2]).drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Valid'])
    df_category2 = df_concat
    count += 1
    
df_concat
'''

In [4]:
unique_clusters_df = pd.read_excel(os.path.join(results_path, 'Final_lower threshold.xlsx'))
unique_clusters = list(set(unique_clusters_df['cluster_id'].to_list()))

In [22]:
#df_all = df_concat[df_concat['cluster_id'].isin(unique_clusters)]
#df_all

In [26]:
'''
product_target = df_all['cluster_id']
product_data = df_all.drop(columns='cluster_id')

# create the train/test split
data_train, data_test, target_train, target_test = train_test_split(product_data, product_target, test_size=0.3, random_state=42, stratify=product_target)
'''

In [33]:
df_table_cleaned = df_cleaned[df_cleaned['table_id'] == 'Product_mountainbikespecialists.com_September2020.json.gz']
table = 'Product_mountainbikespecialists.com_September2020.json.gz'
valid_table_rows = df_table_cleaned['row_id'].to_list()
table_path = cleaned_min3_path
table_df = pd.read_json(os.path.join(table_path, '{}'.format(table)), compression='gzip', lines=True)

In [34]:
table_df

,row_id,name,offers,description,brand,page_url
0,0,Blowtorch,"{'gtin': '708752317437', 'availability': 'https://schema.org/OutOfStock', 'price': '30.00', 'pri...",Sometimes the simplest pieces can be a favorite piece of kit. The Blowtorch neck gaiter is that ...,{'name': '45NRTH'},https://www.mountainbikespecialists.com/product/45nrth-blowtorch-368817-1.htm
1,1,Jekyll 1,"[{'pricecurrency': 'USD', 'price': '8200.00', 'availability': 'https://schema.org/OutOfStock', '...","This precision instrument is for the enduro artists, the ones who will use its Gemini dual-trave...",{'name': 'Cannondale'},https://www.mountainbikespecialists.com/product/cannondale-jekyll-1-345682-1.htm
2,2,K1SL SuperLite Kool Wide Chain,"{'pricecurrency': 'USD', 'price': '25.00', 'gtin': '766759171056', 'availability': 'https://sche...","The KMC K1SL SuperLite Kool Chain is suitable for a variety of single-speed riding styles, such ...",{'name': 'KMC'},https://www.mountainbikespecialists.com/product/kmc-k1sl-superlite-kool-wide-chain-378124-1.htm
3,3,Corkscrew Bottle Opener,"{'pricecurrency': 'USD', 'price': '45.95', 'availability': 'https://schema.org/OutOfStock', 'gti...","In the tradition of our premium quality fold up tools, the BO-4 is a combination corkscrew and b...",{'name': 'Park Tool'},https://www.mountainbikespecialists.com/product/park-tool-corkscrew-bottle-opener-326868-1.htm
4,4,Mountain Sport Organic Disc Brake Pads (Shimano),"{'pricecurrency': 'USD', 'availability': 'https://schema.org/OutOfStock', 'price': '10.99', 'gti...","Jagwire Sport Organic pads feature an all-new resin compound. Mounted on a steel backing plate, ...",{'name': 'Jagwire'},https://www.mountainbikespecialists.com/product/jagwire-mountain-sport-organic-disc-brake-pads-s...
...,...,...,...,...,...,...
1935,1935,Challenge Large Tool Bag,None,The Brooks Challenge Large Tool Bag is a saddle bag for fitment to the rear of a saddle bearing ...,{'name': 'Brooks'},https://www.mountainbikespecialists.com/product/brooks-challenge-large-tool-bag-282611-1.htm
1936,1936,Femto Drive Rear,"{'pricecurrency': 'USD', 'price': '13.49', 'availability': 'https://schema.org/OutOfStock', 'gti...","Lezyne's Femto Drive Rear is a compact, bright, and beautifully made light. The lens is made of ...",{'name': 'Lezyne'},https://www.mountainbikespecialists.com/product/lezyne-femto-drive-rear-192883-1.htm
1937,1937,Impact Pro Women's Mountain Bike Shoe,"[{'pricecurrency': 'USD', 'gtin': '191529206285', 'price': '160.00', 'availability': 'https://sc...",The Impact Pro touts superior toe protection and quicker drying than its predecessor and utilize...,{'name': 'Five Ten'},https://www.mountainbikespecialists.com/product/five-ten-impact-pro-womens-mountain-bike-shoe-37...
1938,1938,Cool Mesh Super Light SL Base Layer,"{'pricecurrency': 'USD', 'gtin': '800550194491', 'availability': 'https://schema.org/OutOfStock'...",Craft's Superlight Mesh top features big-hole design for superior ventilation and cooling. • Mul...,{'name': 'Craft'},https://www.mountainbikespecialists.com/product/craft-cool-mesh-super-light-sl-base-layer-235278...


In [35]:
valid_table_rows

[1718, 1318]

In [36]:
table_df = table_df[table_df['row_id'].isin(valid_table_rows)]
table_df

,row_id,name,offers,description,brand,page_url
1318,1318,Treadwell Neo,"{'pricecurrency': 'USD', 'gtin': '884603786383', 'price': '2300.00', 'availability': 'https://sc...","An urban e-bike for daily rides to class or rehearsal, to the park or the studio, or to meet you...",{'name': 'Cannondale'},https://www.mountainbikespecialists.com/product/cannondale-treadwell-neo-366127-1.htm
1718,1718,Kids Trail Single-Speed 16-inch,"{'pricecurrency': 'USD', 'gtin': '884603785904', 'availability': 'https://schema.org/OutOfStock'...","Get your little riders rolling free and loving life on these high-performance, Kid-Correct bikes...",{'name': 'Cannondale'},https://www.mountainbikespecialists.com/product/cannondale-kids-trail-single-speed-16-inch-36336...


In [37]:
df_table_cleaned

,Unnamed: 0,Unnamed: 0.1,cluster_id,url,row_id,table_id,Valid,name,description,brand_x,brand_y,tokens
9,106,554015,1615000,https://www.mountainbikespecialists.com/product/cannondale-kids-trail-single-speed-16-inch-36336...,1718,Product_mountainbikespecialists.com_September2020.json.gz,1,kids trail single-speed 16-inch,"Get your little riders rolling free and loving life on these high-performance, Kid-Correct bikes...",NaN,NaN,"['kids', 'trail', 'single-speed', '16-inch']"
811,5460,24106971,1001195,https://www.mountainbikespecialists.com/product/cannondale-treadwell-neo-366127-1.htm,1318,Product_mountainbikespecialists.com_September2020.json.gz,1,treadwell neo,"An urban e-bike for daily rides to class or rehearsal, to the park or the studio, or to meet you...",NaN,NaN,"['treadwell', 'neo']"


In [43]:
new_table_df = pd.merge(table_df, df_table_cleaned[['row_id', 'cluster_id', 'table_id', 'brand_x', 'brand_y', 'tokens']], left_on='row_id', right_on='row_id', how='left')

In [44]:
new_table_df

,row_id,name,offers,description,brand,page_url,cluster_id,table_id,brand_x,brand_y,tokens
0,1318,Treadwell Neo,"{'pricecurrency': 'USD', 'gtin': '884603786383', 'price': '2300.00', 'availability': 'https://sc...","An urban e-bike for daily rides to class or rehearsal, to the park or the studio, or to meet you...",{'name': 'Cannondale'},https://www.mountainbikespecialists.com/product/cannondale-treadwell-neo-366127-1.htm,1001195,Product_mountainbikespecialists.com_September2020.json.gz,NaN,NaN,"['treadwell', 'neo']"
1,1718,Kids Trail Single-Speed 16-inch,"{'pricecurrency': 'USD', 'gtin': '884603785904', 'availability': 'https://schema.org/OutOfStock'...","Get your little riders rolling free and loving life on these high-performance, Kid-Correct bikes...",{'name': 'Cannondale'},https://www.mountainbikespecialists.com/product/cannondale-kids-trail-single-speed-16-inch-36336...,1615000,Product_mountainbikespecialists.com_September2020.json.gz,NaN,NaN,"['kids', 'trail', 'single-speed', '16-inch']"


In [45]:
new_table_df['category'] = 'bikes'
new_table_df

,row_id,name,offers,description,brand,page_url,cluster_id,table_id,brand_x,brand_y,tokens,category
0,1318,Treadwell Neo,"{'pricecurrency': 'USD', 'gtin': '884603786383', 'price': '2300.00', 'availability': 'https://sc...","An urban e-bike for daily rides to class or rehearsal, to the park or the studio, or to meet you...",{'name': 'Cannondale'},https://www.mountainbikespecialists.com/product/cannondale-treadwell-neo-366127-1.htm,1001195,Product_mountainbikespecialists.com_September2020.json.gz,NaN,NaN,"['treadwell', 'neo']",bikes
1,1718,Kids Trail Single-Speed 16-inch,"{'pricecurrency': 'USD', 'gtin': '884603785904', 'availability': 'https://schema.org/OutOfStock'...","Get your little riders rolling free and loving life on these high-performance, Kid-Correct bikes...",{'name': 'Cannondale'},https://www.mountainbikespecialists.com/product/cannondale-kids-trail-single-speed-16-inch-36336...,1615000,Product_mountainbikespecialists.com_September2020.json.gz,NaN,NaN,"['kids', 'trail', 'single-speed', '16-inch']",bikes


In [46]:
new_table_df.to_json(os.path.join(train_test_input_filtered_path, '{}'.format(entity.lower()), '{}'.format(table)), compression='gzip', orient='records', lines=True)

## code

In [54]:
entities = ['Bikes', 'Cars', 'Clothes', 'Drugstore', 'Electronics', 'Technology', 'Tools', 'Random']

files_dict = {'minimum3': [file for file in os.listdir(min3_path) if file.endswith('.json.gz')], 
             'top100': [file for file in os.listdir(top100_path) if file.endswith('.json.gz')]}
    
for entity in entities:
    print(entity)
    df_category = pd.read_csv(os.path.join(cluster_path,
                                                '{}_clusters_all_8_tables_post_processed_lower_threshold.csv'.format(
                                                    entity)))
    df_cleaned = df_category[df_category['cluster_id'].isin(unique_clusters)]
    valid_tables = list(set(df_cleaned['table_id'].to_list()))
    
    count = 0
    with progressbar.ProgressBar(max_value=len(valid_tables)) as bar:
        for table in valid_tables:
            count += 1
            if table in files_dict['minimum3']:
                table_path = min3_path
            elif table in files_dict['top100']:
                table_path = top100_path
            else:
                print('no valid path for: {}'.format(table))
                continue

            df_table_cleaned = df_cleaned[df_cleaned['table_id'] == table]
            valid_table_rows = df_table_cleaned['row_id'].to_list()
            table_df = pd.read_json(os.path.join(table_path, '{}'.format(table)), compression='gzip', lines=True)
            table_df = table_df[table_df['row_id'].isin(valid_table_rows)]
            new_table_df = pd.merge(table_df, df_table_cleaned[['row_id', 'cluster_id', 'table_id', 'brand_x', 'brand_y', 'tokens']], left_on='row_id', right_on='row_id', how='left')
            new_table_df['category'] = entity.lower()
            new_table_df.to_json(os.path.join(train_test_input_filtered_path, '{}'.format(entity.lower()), '{}'.format(table)), compression='gzip', orient='records', lines=True)
            
            bar.update(count)
            

  0% (0 of 126) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--

Bikes


100% (126 of 126) |######################| Elapsed Time: 0:00:12 Time:  0:00:12


Cars


100% (234 of 234) |######################| Elapsed Time: 0:00:25 Time:  0:00:25


...

100% (476 of 476) |######################| Elapsed Time: 0:00:26 Time:  0:00:26


Tools


100% (9 of 9) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


Random


100% (2280 of 2280) |####################| Elapsed Time: 0:03:11 Time:  0:03:11


In [55]:
input_files_dict = {}
for entity in entities:
    train_test_category_path = os.path.join(train_test_input_filtered_path, '{}'.format(entity.lower()))
    input_files_dict[entity.lower()] = [file for file in os.listdir(train_test_category_path) if file.endswith('.json.gz')]

## overlap between two categories

In [66]:
# try to find overlap between two categories:
combinations_list = []
for pair in itertools.combinations(entities,2):
    combinations_list.append(pair)
    
overlap_dict = {}
for pair in combinations_list:
    overlap = list(set(input_files_dict[pair[0].lower()]) & set(input_files_dict[pair[1].lower()]))
    overlap_dict[str(pair)] = overlap
    
overlap_dict

# save overlap_dict
with open(os.path.join(train_test_input_filtered_path, 'all/overlap_tables_between_categories_dict.json'), 'w', encoding='utf-8') as f:
    json.dump(overlap_dict, f)

In [67]:
# save all tables in one path
for entity in entities:
    print(entity)
    train_test_category_path = os.path.join(train_test_input_filtered_path, '{}'.format(entity.lower()))
    category_tables_list = [file for file in os.listdir(train_test_category_path) if file.endswith('.json.gz')]
    
    count = 0
    with progressbar.ProgressBar(max_value=len(valid_tables)) as bar:
        for table in category_tables_list:
            if os.path.isfile(os.path.join(train_test_all_filtered_path,'{}'.format(table))):
                all_table_df = pd.read_json(os.path.join(train_test_all_filtered_path,'{}'.format(table)), compression='gzip', lines=True)
                cat_table_df = pd.read_json(os.path.join(train_test_category_path,'{}'.format(table)), compression='gzip', lines=True)
                frames = [all_table_df, cat_table_df]
                all_table_df = pd.concat(frames)
            else:
                all_table_df = pd.read_json(os.path.join(train_test_category_path,'{}'.format(table)), compression='gzip', lines=True)
            all_table_df.to_json(os.path.join(train_test_all_filtered_path, '{}'.format(table)), compression='gzip', orient='records', lines=True)
            
            count += 1
            bar.update(count)

  0% (13 of 2280) |                      | Elapsed Time: 0:00:00 ETA:   0:00:18

Bikes


100% (2280 of 2280) |####################| Elapsed Time: 0:00:00 Time:  0:00:00
  1% (29 of 2280) |                      | Elapsed Time: 0:00:00 ETA:   0:00:14

Cars


100% (2280 of 2280) |####################| Elapsed Time: 0:00:01 Time:  0:00:01
  0% (16 of 2280) |                      | Elapsed Time: 0:00:00 ETA:   0:00:14

...

100% (2280 of 2280) |####################| Elapsed Time: 0:00:19 Time:  0:00:19
  0% (3 of 2280) |                       | Elapsed Time: 0:00:00 ETA:   0:01:21

Technology


100% (2280 of 2280) |####################| Elapsed Time: 0:00:08 Time:  0:00:08
100% (2280 of 2280) |####################| Elapsed Time: 0:00:00 Time:  0:00:00


Tools
Random


100% (2280 of 2280) |####################| Elapsed Time: 0:00:28 Time:  0:00:28


## Concatenate all filtered tables into one dataframe

In [10]:
all_filtered_tables = [file for file in os.listdir(train_test_all_filtered_path) if file.endswith('.json.gz')]
if 'train_test_all_filtered_tables.json.gz' in all_filtered_tables:
    all_filtered_tables.remove('train_test_all_filtered_tables.json.gz')

count = 0
with progressbar.ProgressBar(max_value=len(all_filtered_tables)) as bar:
    print('running concatenation')
    for table in all_filtered_tables:
        table_df_1 = pd.read_json(os.path.join(train_test_all_filtered_path,'{}'.format(table)), compression='gzip', lines=True)
        if count == 0:
            df_concat = table_df_1
        if count > 0:
            df_concat = pd.concat([table_df_1, table_df_2])
        table_df_2 = df_concat
        if count == len(all_filtered_tables) - 1:
        count += 1
        bar.update(count)
    
df_concat.to_json(os.path.join(train_test_all_filtered_path, 'train_test_all_filtered_tables.json.gz'), compression='gzip', orient='records', lines=True)
df_concat

  0% (8 of 4356) |                       | Elapsed Time: 0:00:00 ETA:   0:00:55

running concatenation


100% (4356 of 4356) |####################| Elapsed Time: 0:30:31 Time:  0:30:31


,row_id,name,offers,description,sku,brand,page_url,cluster_id,table_id,brand_x,...,video,position,ratingcount,target,breadcrumb,ispartof,speakable,title,isconsumablefor,productbrand
0,399,WD 2TB Elements Portable External Hard Drive - USB 3.0 - WDBU6Y0020BBK-WESN,"{'pricecurrency': 'USD', 'price': '74.99', 'sku': 'B06W55K9N6', 'availability': 'http://schema.o...",Brand: Western Digital Color: black Features: USB 3.0 and USB 2.0 compatibility Fast data transf...,B06W55K9N6,{'name': 'Western Digital'},https://area399.com/products/wd-2tb-elements-portable-external-hard-drive-usb-3-0-wdbu6y0020bbk-...,541658,Product_area399.com_September2020.json.gz,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,18,Three Gifts Goat Milk Soap,"{'price': '8.5', 'availability': 'https://schema.org/InStock', 'pricecurrency': 'USD'}","Gold, frankincense and myrrh - three gifts given from Kings to a baby who fulfilled all the prop...",NaN,NaN,https://www.thecraftist.com/product-page/three-gifts-goat-milk-soap,65383188,Product_thecraftist.com_September2020.json.gz,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,601,GoPro Head Strap + Quick Clip,"{'pricecurrency': 'CAD', 'availability': 'https://schema.org/InStock', 'validfrom': '2020-09-23'...",NaN,NaN,GoPro,https://www.specialisteduski.com/gopro-head-strap-quick-clip.html,863679,Product_specialisteduski.com_September2020.json.gz,GoPro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,722,GoPro 3 way grip Arm GoPro,"{'price': '102.99', 'pricecurrency': 'CAD', 'availability': 'https://schema.org/InStock', 'price...",NaN,NaN,GoPro,https://www.specialisteduski.com/gopro-3-way-grip-arm-gopro.html,251884,Product_specialisteduski.com_September2020.json.gz,GoPro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,mason pearson brush.,"{'pricecurrency': 'USD', 'price': '175.00', 'itemcondition': 'http://schema.org/NewCondition', '...",A handcrafted hairbrush from Mason Pearson with a smooth contoured handle and soft rubber cushio...,210000000083,NaN,https://www.malinandgoetz.com/mason-pearson-brush,1617106,Product_malinandgoetz.com_September2020.json.gz,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,834,Gibson Les Paul Joe Bonamassa Goldtop,"{'price': '2599.00', 'pricevaliduntil': '2021-12-31', 'availability': 'http://schema.org/OutOfSt...",NaN,210000000448.0,NaN,https://sixtysixtysounds.com/product/gibson-les-paul-joe-bonamassa-goldtop/,1257145,Product_sixtysixtysounds.com_September2020.json.gz,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,854,Gibson Birdseye Les Paul 1959 Cherry,"{'price': '0.00', 'pricevaliduntil': '2021-12-31', 'availability': 'http://schema.org/OutOfStock...",NaN,210000000722.0,NaN,https://sixtysixtysounds.com/product/gibson-birdseye-les-paul-1959-cherry/,1524820,Product_sixtysixtysounds.com_September2020.json.gz,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,442,Epiphone Masterbilt AJ 500 RCE,"{'pricevaliduntil': '2021-12-31', 'price': '449.00', 'pricecurrency': 'GBP', 'availability': 'ht...",NaN,210000000077.0,NaN,https://sixtysixtysounds.com/product/epiphone-masterbilt-aj-500-rce/,1702641,Product_sixtysixtysounds.com_September2020.json.gz,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1074,Essência White Beer 500g,"{'price': 'R$ 92,00'}",NaN,NaN,NaN,https://www.lojapeterpaiva.com.br/produto/essencia-white-beer-500g/3338513,2434654,Product_lojapeterpaiva.com.br_September2020.json.gz,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Peter Paiva


In [3]:
all_filtered_tables_df = pd.read_json(os.path.join(train_test_all_filtered_path,'train_test_all_filtered_tables.json.gz'), compression='gzip', lines=True)

In [12]:
all_filtered_tables_df = all_filtered_tables_df.loc[all_filtered_tables_df.astype(str).drop_duplicates().index]
all_filtered_tables_df.to_json(os.path.join(train_test_all_filtered_path, 'train_test_all_filtered_tables.json.gz'), compression='gzip', orient='records', lines=True)
all_filtered_tables_df

,row_id,name,offers,description,sku,brand,page_url,cluster_id,table_id,brand_x,...,video,position,ratingcount,target,breadcrumb,ispartof,speakable,title,isconsumablefor,productbrand
0,399,WD 2TB Elements Portable External Hard Drive - USB 3.0 - WDBU6Y0020BBK-WESN,"{'pricecurrency': 'USD', 'price': '74.99', 'sku': 'B06W55K9N6', 'availability': 'http://schema.o...",Brand: Western Digital Color: black Features: USB 3.0 and USB 2.0 compatibility Fast data transf...,B06W55K9N6,{'name': 'Western Digital'},https://area399.com/products/wd-2tb-elements-portable-external-hard-drive-usb-3-0-wdbu6y0020bbk-...,541658,Product_area399.com_September2020.json.gz,None,...,None,NaN,NaN,None,None,None,None,None,None,None
1,18,Three Gifts Goat Milk Soap,"{'price': '8.5', 'availability': 'https://schema.org/InStock', 'pricecurrency': 'USD'}","Gold, frankincense and myrrh - three gifts given from Kings to a baby who fulfilled all the prop...",None,None,https://www.thecraftist.com/product-page/three-gifts-goat-milk-soap,65383188,Product_thecraftist.com_September2020.json.gz,None,...,None,NaN,NaN,None,None,None,None,None,None,None
2,601,GoPro Head Strap + Quick Clip,"{'pricecurrency': 'CAD', 'availability': 'https://schema.org/InStock', 'validfrom': '2020-09-23'...",None,None,GoPro,https://www.specialisteduski.com/gopro-head-strap-quick-clip.html,863679,Product_specialisteduski.com_September2020.json.gz,GoPro,...,None,NaN,NaN,None,None,None,None,None,None,None
3,722,GoPro 3 way grip Arm GoPro,"{'price': '102.99', 'pricecurrency': 'CAD', 'availability': 'https://schema.org/InStock', 'price...",None,None,GoPro,https://www.specialisteduski.com/gopro-3-way-grip-arm-gopro.html,251884,Product_specialisteduski.com_September2020.json.gz,GoPro,...,None,NaN,NaN,None,None,None,None,None,None,None
4,0,mason pearson brush.,"{'pricecurrency': 'USD', 'price': '175.00', 'itemcondition': 'http://schema.org/NewCondition', '...",A handcrafted hairbrush from Mason Pearson with a smooth contoured handle and soft rubber cushio...,210000000083,None,https://www.malinandgoetz.com/mason-pearson-brush,1617106,Product_malinandgoetz.com_September2020.json.gz,None,...,None,NaN,NaN,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79604,834,Gibson Les Paul Joe Bonamassa Goldtop,"{'price': '2599.00', 'pricevaliduntil': '2021-12-31', 'availability': 'http://schema.org/OutOfSt...",None,210000000448.0,None,https://sixtysixtysounds.com/product/gibson-les-paul-joe-bonamassa-goldtop/,1257145,Product_sixtysixtysounds.com_September2020.json.gz,None,...,None,NaN,NaN,None,None,None,None,None,None,None
79605,854,Gibson Birdseye Les Paul 1959 Cherry,"{'price': '0.00', 'pricevaliduntil': '2021-12-31', 'availability': 'http://schema.org/OutOfStock...",None,210000000722.0,None,https://sixtysixtysounds.com/product/gibson-birdseye-les-paul-1959-cherry/,1524820,Product_sixtysixtysounds.com_September2020.json.gz,None,...,None,NaN,NaN,None,None,None,None,None,None,None
79606,442,Epiphone Masterbilt AJ 500 RCE,"{'pricevaliduntil': '2021-12-31', 'price': '449.00', 'pricecurrency': 'GBP', 'availability': 'ht...",None,210000000077.0,None,https://sixtysixtysounds.com/product/epiphone-masterbilt-aj-500-rce/,1702641,Product_sixtysixtysounds.com_September2020.json.gz,None,...,None,NaN,NaN,None,None,None,None,None,None,None
79607,1074,Essência White Beer 500g,"{'price': 'R$ 92,00'}",None,None,None,https://www.lojapeterpaiva.com.br/produto/essencia-white-beer-500g/3338513,2434654,Product_lojapeterpaiva.com.br_September2020.json.gz,None,...,None,NaN,NaN,None,None,None,None,None,None,Peter Paiva


# Generate unfiltered tables

### testing

In [14]:
files_dict = {'minimum3': [file for file in os.listdir(min3_path) if file.endswith('.json.gz')], 
             'top100': [file for file in os.listdir(top100_path) if file.endswith('.json.gz')]}
    
table = 'Product_1clicksafety.com_September2020.json.gz'
if table in files_dict['minimum3']:
    table_path = min3_path
elif table in files_dict['top100']:
    table_path = top100_path
else:
    print('no valid path for: {}'.format(table))

filtered_table_df = pd.read_json(os.path.join(train_test_all_filtered_path, '{}'.format(table)), compression='gzip', lines=True)
filtered_row_ids = filtered_table_df['row_id'].to_list()
original_table_df = pd.read_json(os.path.join(table_path, '{}'.format(table)), compression='gzip', lines=True)

In [16]:
filtered_table_df

,row_id,name,offers,description,sku,url,page_url,cluster_id,table_id,brand_x,brand_y,tokens,category
0,173,Mini Alert Infrared Alarm,"{'pricecurrency': 'USD', 'availability': 'http://schema.org/InStock', 'pricevaliduntil': '2021-1...",The MINI ALERT ALARM uses passive infrared technology to protect a room from intruders. Producin...,MINI2040,{'name': 'Mini Alert Infrared Alarm'},https://1clicksafety.com/self-defense/mini-alert-infrared-alarm/,58254847,Product_1clicksafety.com_September2020.json.gz,NaN,mini,"['mini', 'alert', 'infrared', 'alarm']",cars


In [17]:
original_table_df

,row_id,name,offers,description,sku,url,page_url
0,0,Pepper Shot 1.2% MC 1/2 oz Lipstick Pepper Sprays,"{'pricecurrency': 'USD', 'availability': 'http://schema.org/InStock', 'pricevaliduntil': '2021-1...",Pepper Shot 1.2% MC ½ oz lipstick pepper spray black. It's designed for women. Pepper Shot Peppe...,5119,{'name': 'Pepper Shot 1.2% MC 1/2 oz Lipstick Pepper Sprays'},https://1clicksafety.com/self-defense/pepper-shot-lipstick-pepper-spray/
1,1,Taser Pulse Soft Holster,"{'pricecurrency': 'USD', 'availability': 'http://schema.org/InStock', 'price': '14.99', 'priceva...",Carry your TASER Pulse/Pulse+ easily with this outside the waistband holster. The lightweight an...,39062,{'name': 'Taser Pulse Soft Holster'},https://1clicksafety.com/self-defense/taser-pulse-soft-holster/
2,2,Kubotan Aircraft Grade Aluminum Keychain,"{'price': '10.00', 'availability': 'http://schema.org/InStock', 'pricevaliduntil': '2021-12-31',...",Aircraft Grade Aluminum Keychain Kubotan. Grip your kubotan and get to protecting yourself. A gr...,5044,{'name': 'Kubotan Aircraft Grade Aluminum Keychain'},https://1clicksafety.com/self-defense/kubotan-aircraft-grade-aluminum-keychain/
3,3,Glass Breakage Alarm,"{'pricecurrency': 'USD', 'availability': 'http://schema.org/InStock', 'price': '9.95', 'priceval...",The GLASS BREAKAGE ALARM is an effective low cost vibration alarm that can be used for securing ...,HP-GLASS,{'name': 'Glass Breakage Alarm'},https://1clicksafety.com/self-defense/glass-breakage-alarm/
4,4,Dummy Dome Camera With LED,"{'pricecurrency': 'USD', 'availability': 'http://schema.org/InStock', 'pricevaliduntil': '2021-1...",The Dome Dummy Camera with flashing LED Features a motion detection LED flashing light. Does not...,DM-DOML,{'name': 'Dummy Dome Camera With LED'},https://1clicksafety.com/self-defense/dummy-dome-camera-led/
...,...,...,...,...,...,...,...
185,185,Solar Powered Dummy Camera,"{'pricecurrency': 'USD', 'availability': 'http://schema.org/InStock', 'pricevaliduntil': '2021-1...",Solar Powered Dummy Camera made with a quality ABS plastic. The Dummy camera has a flashing red ...,4943,{'name': 'Solar Powered Dummy Camera'},https://1clicksafety.com/self-defense/solar-powered-dummy-camera/
186,186,Stun Gun Knife and Flashlight,"{'pricecurrency': 'USD', 'availability': 'http://schema.org/InStock', 'pricevaliduntil': '2021-1...","Stun Knife 20 Million volt Stun Gun. Comfortable in hand, the Stun Knife delivers super powered ...",ST-STUNKNIFE,{'name': 'Stun Gun Knife and Flashlight'},https://1clicksafety.com/self-defense/stun-gun-knife-flashlight/
187,187,Wildfire™ 1.4% MC Pepper Spray Fogger,"{'highprice': '59.95', 'pricecurrency': 'USD', 'lowprice': '24.95', 'availability': 'http://sche...",Wildfire™ 2 oz 1.4% MC flip top actuator pepper spray stream. The Major Capsaicinoids are the tr...,5288,{'name': 'Wildfire™ 1.4% MC Pepper Spray Fogger'},https://1clicksafety.com/self-defense/wildfire-pepper-spray-fogger/
188,188,Spray Bottle Diversion Safe,"{'pricecurrency': 'USD', 'availability': 'http://schema.org/InStock', 'pricevaliduntil': '2021-1...","Spray Bottle Diversion Safe Interior dimensions of safe 1¾\"" x 5¼\",DS-SPRAY,{'name': 'Spray Bottle Diversion Safe'},https://1clicksafety.com/self-defense/spray-bottle-hidden-safe/


In [23]:
original_table_without_df = original_table_df[~original_table_df['row_id'].isin(filtered_row_ids)]
original_table_without_df

,row_id,name,offers,description,sku,url,page_url
0,0,Pepper Shot 1.2% MC 1/2 oz Lipstick Pepper Sprays,"{'pricecurrency': 'USD', 'availability': 'http://schema.org/InStock', 'pricevaliduntil': '2021-1...",Pepper Shot 1.2% MC ½ oz lipstick pepper spray black. It's designed for women. Pepper Shot Peppe...,5119,{'name': 'Pepper Shot 1.2% MC 1/2 oz Lipstick Pepper Sprays'},https://1clicksafety.com/self-defense/pepper-shot-lipstick-pepper-spray/
1,1,Taser Pulse Soft Holster,"{'pricecurrency': 'USD', 'availability': 'http://schema.org/InStock', 'price': '14.99', 'priceva...",Carry your TASER Pulse/Pulse+ easily with this outside the waistband holster. The lightweight an...,39062,{'name': 'Taser Pulse Soft Holster'},https://1clicksafety.com/self-defense/taser-pulse-soft-holster/
2,2,Kubotan Aircraft Grade Aluminum Keychain,"{'price': '10.00', 'availability': 'http://schema.org/InStock', 'pricevaliduntil': '2021-12-31',...",Aircraft Grade Aluminum Keychain Kubotan. Grip your kubotan and get to protecting yourself. A gr...,5044,{'name': 'Kubotan Aircraft Grade Aluminum Keychain'},https://1clicksafety.com/self-defense/kubotan-aircraft-grade-aluminum-keychain/
3,3,Glass Breakage Alarm,"{'pricecurrency': 'USD', 'availability': 'http://schema.org/InStock', 'price': '9.95', 'priceval...",The GLASS BREAKAGE ALARM is an effective low cost vibration alarm that can be used for securing ...,HP-GLASS,{'name': 'Glass Breakage Alarm'},https://1clicksafety.com/self-defense/glass-breakage-alarm/
4,4,Dummy Dome Camera With LED,"{'pricecurrency': 'USD', 'availability': 'http://schema.org/InStock', 'pricevaliduntil': '2021-1...",The Dome Dummy Camera with flashing LED Features a motion detection LED flashing light. Does not...,DM-DOML,{'name': 'Dummy Dome Camera With LED'},https://1clicksafety.com/self-defense/dummy-dome-camera-led/
...,...,...,...,...,...,...,...
185,185,Solar Powered Dummy Camera,"{'pricecurrency': 'USD', 'availability': 'http://schema.org/InStock', 'pricevaliduntil': '2021-1...",Solar Powered Dummy Camera made with a quality ABS plastic. The Dummy camera has a flashing red ...,4943,{'name': 'Solar Powered Dummy Camera'},https://1clicksafety.com/self-defense/solar-powered-dummy-camera/
186,186,Stun Gun Knife and Flashlight,"{'pricecurrency': 'USD', 'availability': 'http://schema.org/InStock', 'pricevaliduntil': '2021-1...","Stun Knife 20 Million volt Stun Gun. Comfortable in hand, the Stun Knife delivers super powered ...",ST-STUNKNIFE,{'name': 'Stun Gun Knife and Flashlight'},https://1clicksafety.com/self-defense/stun-gun-knife-flashlight/
187,187,Wildfire™ 1.4% MC Pepper Spray Fogger,"{'highprice': '59.95', 'pricecurrency': 'USD', 'lowprice': '24.95', 'availability': 'http://sche...",Wildfire™ 2 oz 1.4% MC flip top actuator pepper spray stream. The Major Capsaicinoids are the tr...,5288,{'name': 'Wildfire™ 1.4% MC Pepper Spray Fogger'},https://1clicksafety.com/self-defense/wildfire-pepper-spray-fogger/
188,188,Spray Bottle Diversion Safe,"{'pricecurrency': 'USD', 'availability': 'http://schema.org/InStock', 'pricevaliduntil': '2021-1...","Spray Bottle Diversion Safe Interior dimensions of safe 1¾\"" x 5¼\",DS-SPRAY,{'name': 'Spray Bottle Diversion Safe'},https://1clicksafety.com/self-defense/spray-bottle-hidden-safe/


In [25]:
original_table_without_df = pd.concat([original_table_without_df, pd.DataFrame(columns=list(['row_id', 'cluster_id', 'table_id', 'brand_x', 'brand_y', 'tokens', 'category']))])
original_table_without_df                                       

,row_id,name,offers,description,sku,url,page_url,cluster_id,table_id,brand_x,brand_y,tokens,category
0,0,Pepper Shot 1.2% MC 1/2 oz Lipstick Pepper Sprays,"{'pricecurrency': 'USD', 'availability': 'http://schema.org/InStock', 'pricevaliduntil': '2021-1...",Pepper Shot 1.2% MC ½ oz lipstick pepper spray black. It's designed for women. Pepper Shot Peppe...,5119,{'name': 'Pepper Shot 1.2% MC 1/2 oz Lipstick Pepper Sprays'},https://1clicksafety.com/self-defense/pepper-shot-lipstick-pepper-spray/,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Taser Pulse Soft Holster,"{'pricecurrency': 'USD', 'availability': 'http://schema.org/InStock', 'price': '14.99', 'priceva...",Carry your TASER Pulse/Pulse+ easily with this outside the waistband holster. The lightweight an...,39062,{'name': 'Taser Pulse Soft Holster'},https://1clicksafety.com/self-defense/taser-pulse-soft-holster/,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Kubotan Aircraft Grade Aluminum Keychain,"{'price': '10.00', 'availability': 'http://schema.org/InStock', 'pricevaliduntil': '2021-12-31',...",Aircraft Grade Aluminum Keychain Kubotan. Grip your kubotan and get to protecting yourself. A gr...,5044,{'name': 'Kubotan Aircraft Grade Aluminum Keychain'},https://1clicksafety.com/self-defense/kubotan-aircraft-grade-aluminum-keychain/,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Glass Breakage Alarm,"{'pricecurrency': 'USD', 'availability': 'http://schema.org/InStock', 'price': '9.95', 'priceval...",The GLASS BREAKAGE ALARM is an effective low cost vibration alarm that can be used for securing ...,HP-GLASS,{'name': 'Glass Breakage Alarm'},https://1clicksafety.com/self-defense/glass-breakage-alarm/,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Dummy Dome Camera With LED,"{'pricecurrency': 'USD', 'availability': 'http://schema.org/InStock', 'pricevaliduntil': '2021-1...",The Dome Dummy Camera with flashing LED Features a motion detection LED flashing light. Does not...,DM-DOML,{'name': 'Dummy Dome Camera With LED'},https://1clicksafety.com/self-defense/dummy-dome-camera-led/,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,185,Solar Powered Dummy Camera,"{'pricecurrency': 'USD', 'availability': 'http://schema.org/InStock', 'pricevaliduntil': '2021-1...",Solar Powered Dummy Camera made with a quality ABS plastic. The Dummy camera has a flashing red ...,4943,{'name': 'Solar Powered Dummy Camera'},https://1clicksafety.com/self-defense/solar-powered-dummy-camera/,NaN,NaN,NaN,NaN,NaN,NaN
186,186,Stun Gun Knife and Flashlight,"{'pricecurrency': 'USD', 'availability': 'http://schema.org/InStock', 'pricevaliduntil': '2021-1...","Stun Knife 20 Million volt Stun Gun. Comfortable in hand, the Stun Knife delivers super powered ...",ST-STUNKNIFE,{'name': 'Stun Gun Knife and Flashlight'},https://1clicksafety.com/self-defense/stun-gun-knife-flashlight/,NaN,NaN,NaN,NaN,NaN,NaN
187,187,Wildfire™ 1.4% MC Pepper Spray Fogger,"{'highprice': '59.95', 'pricecurrency': 'USD', 'lowprice': '24.95', 'availability': 'http://sche...",Wildfire™ 2 oz 1.4% MC flip top actuator pepper spray stream. The Major Capsaicinoids are the tr...,5288,{'name': 'Wildfire™ 1.4% MC Pepper Spray Fogger'},https://1clicksafety.com/self-defense/wildfire-pepper-spray-fogger/,NaN,NaN,NaN,NaN,NaN,NaN
188,188,Spray Bottle Diversion Safe,"{'pricecurrency': 'USD', 'availability': 'http://schema.org/InStock', 'pricevaliduntil': '2021-1...","Spray Bottle Diversion Safe Interior dimensions of safe 1¾\"" x 5¼\",DS-SPRAY,{'name': 'Spray Bottle Diversion Safe'},https://1clicksafety.com/self-defense/spray-bottle-hidden-safe/,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
new_table_df = pd.concat([filtered_table_df, original_table_without_df])
new_table_df

,row_id,name,offers,description,sku,url,page_url,cluster_id,table_id,brand_x,brand_y,tokens,category
0,173,Mini Alert Infrared Alarm,"{'pricecurrency': 'USD', 'availability': 'http://schema.org/InStock', 'pricevaliduntil': '2021-1...",The MINI ALERT ALARM uses passive infrared technology to protect a room from intruders. Producin...,MINI2040,{'name': 'Mini Alert Infrared Alarm'},https://1clicksafety.com/self-defense/mini-alert-infrared-alarm/,58254847,Product_1clicksafety.com_September2020.json.gz,NaN,mini,"['mini', 'alert', 'infrared', 'alarm']",cars
0,0,Pepper Shot 1.2% MC 1/2 oz Lipstick Pepper Sprays,"{'pricecurrency': 'USD', 'availability': 'http://schema.org/InStock', 'pricevaliduntil': '2021-1...",Pepper Shot 1.2% MC ½ oz lipstick pepper spray black. It's designed for women. Pepper Shot Peppe...,5119,{'name': 'Pepper Shot 1.2% MC 1/2 oz Lipstick Pepper Sprays'},https://1clicksafety.com/self-defense/pepper-shot-lipstick-pepper-spray/,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Taser Pulse Soft Holster,"{'pricecurrency': 'USD', 'availability': 'http://schema.org/InStock', 'price': '14.99', 'priceva...",Carry your TASER Pulse/Pulse+ easily with this outside the waistband holster. The lightweight an...,39062,{'name': 'Taser Pulse Soft Holster'},https://1clicksafety.com/self-defense/taser-pulse-soft-holster/,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Kubotan Aircraft Grade Aluminum Keychain,"{'price': '10.00', 'availability': 'http://schema.org/InStock', 'pricevaliduntil': '2021-12-31',...",Aircraft Grade Aluminum Keychain Kubotan. Grip your kubotan and get to protecting yourself. A gr...,5044,{'name': 'Kubotan Aircraft Grade Aluminum Keychain'},https://1clicksafety.com/self-defense/kubotan-aircraft-grade-aluminum-keychain/,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Glass Breakage Alarm,"{'pricecurrency': 'USD', 'availability': 'http://schema.org/InStock', 'price': '9.95', 'priceval...",The GLASS BREAKAGE ALARM is an effective low cost vibration alarm that can be used for securing ...,HP-GLASS,{'name': 'Glass Breakage Alarm'},https://1clicksafety.com/self-defense/glass-breakage-alarm/,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,185,Solar Powered Dummy Camera,"{'pricecurrency': 'USD', 'availability': 'http://schema.org/InStock', 'pricevaliduntil': '2021-1...",Solar Powered Dummy Camera made with a quality ABS plastic. The Dummy camera has a flashing red ...,4943,{'name': 'Solar Powered Dummy Camera'},https://1clicksafety.com/self-defense/solar-powered-dummy-camera/,NaN,NaN,NaN,NaN,NaN,NaN
186,186,Stun Gun Knife and Flashlight,"{'pricecurrency': 'USD', 'availability': 'http://schema.org/InStock', 'pricevaliduntil': '2021-1...","Stun Knife 20 Million volt Stun Gun. Comfortable in hand, the Stun Knife delivers super powered ...",ST-STUNKNIFE,{'name': 'Stun Gun Knife and Flashlight'},https://1clicksafety.com/self-defense/stun-gun-knife-flashlight/,NaN,NaN,NaN,NaN,NaN,NaN
187,187,Wildfire™ 1.4% MC Pepper Spray Fogger,"{'highprice': '59.95', 'pricecurrency': 'USD', 'lowprice': '24.95', 'availability': 'http://sche...",Wildfire™ 2 oz 1.4% MC flip top actuator pepper spray stream. The Major Capsaicinoids are the tr...,5288,{'name': 'Wildfire™ 1.4% MC Pepper Spray Fogger'},https://1clicksafety.com/self-defense/wildfire-pepper-spray-fogger/,NaN,NaN,NaN,NaN,NaN,NaN
188,188,Spray Bottle Diversion Safe,"{'pricecurrency': 'USD', 'availability': 'http://schema.org/InStock', 'pricevaliduntil': '2021-1...","Spray Bottle Diversion Safe Interior dimensions of safe 1¾\"" x 5¼\",DS-SPRAY,{'name': 'Spray Bottle Diversion Safe'},https://1clicksafety.com/self-defense/spray-bottle-hidden-safe/,NaN,NaN,NaN,NaN,NaN,NaN


### code

In [29]:
all_filtered_tables = [file for file in os.listdir(train_test_all_filtered_path) if file.endswith('.json.gz')]
if 'train_test_all_filtered_tables.json.gz' in all_filtered_tables:
    all_filtered_tables.remove('train_test_all_filtered_tables.json.gz')
    
files_dict = {'minimum3': [file for file in os.listdir(min3_path) if file.endswith('.json.gz')], 
             'top100': [file for file in os.listdir(top100_path) if file.endswith('.json.gz')]}
    
count = 0
with progressbar.ProgressBar(max_value=len(all_filtered_tables)) as bar:
    for table in all_filtered_tables:
        count += 1
        if table in files_dict['minimum3']:
            table_path = min3_path
        elif table in files_dict['top100']:
            table_path = top100_path
        else:
            print('no valid path for: {}'.format(table))
            continue

        filtered_table_df = pd.read_json(os.path.join(train_test_all_filtered_path, '{}'.format(table)), compression='gzip', lines=True)
        filtered_row_ids = filtered_table_df['row_id'].to_list()
        original_table_df = pd.read_json(os.path.join(table_path, '{}'.format(table)), compression='gzip', lines=True)
        original_table_without_df = original_table_df[~original_table_df['row_id'].isin(filtered_row_ids)]
        new_table_df = pd.concat([filtered_table_df, original_table_without_df])
        new_table_df.to_json(os.path.join(train_test_input_unfiltered_path, '{}'.format(table)), compression='gzip', orient='records', lines=True)

        bar.update(count)

100% (4355 of 4355) |####################| Elapsed Time: 0:11:59 Time:  0:11:59


# Concatenate all unfiltered tables into one dataframe

In [30]:
all_unfiltered_tables = [file for file in os.listdir(train_test_input_unfiltered_path) if file.endswith('.json.gz')]
if 'train_test_all_unfiltered_tables.json.gz' in all_unfiltered_tables:
    all_filtered_tables.remove('train_test_all_unfiltered_tables.json.gz')

count = 0
with progressbar.ProgressBar(max_value=len(all_filtered_tables)) as bar:
    print('running concatenation')
    for table in all_unfiltered_tables:
        table_df_1 = pd.read_json(os.path.join(train_test_input_unfiltered_path,'{}'.format(table)), compression='gzip', lines=True)
        if count == 0:
            df_concat = table_df_1
        if count > 0:
            df_concat = pd.concat([table_df_1, table_df_2])
        table_df_2 = df_concat
        count += 1
        bar.update(count)
    
df_concat.to_json(os.path.join(train_test_input_unfiltered_path, 'train_test_all_unfiltered_tables.json.gz'), compression='gzip', orient='records', lines=True)
df_concat

  0% (5 of 4355) |                       | Elapsed Time: 0:00:00 ETA:   0:01:31

running concatenation


 16% (699 of 4355) |###                  | Elapsed Time: 0:12:47 ETA:   2:23:13


KeyboardInterrupt: 

# Create train test split

In [ ]:
#all_tables = pd.read_json(os.path.join(train_test_input_unfiltered_path,'train_test_all_unfiltered_tables.json.gz'), compression='gzip', lines=True)

## Create overview table

In [13]:
all_unfiltered_tables = [file for file in os.listdir(train_test_input_unfiltered_path) if file.endswith('.json.gz')]
if 'train_test_all_unfiltered_tables.json.gz' in all_unfiltered_tables:
    all_filtered_tables.remove('train_test_all_unfiltered_tables.json.gz')

In [25]:
overview_df = pd.DataFrame()
overview_df['table_id'] = all_unfiltered_tables

cluster_id_overview_list = []
count = 0
with progressbar.ProgressBar(max_value=len(all_unfiltered_tables)) as bar:
    for table in all_unfiltered_tables:
        table_df = pd.read_json(os.path.join(train_test_input_unfiltered_path,'{}'.format(table)), compression='gzip', lines=True)
        table_df['cluster_id'] = table_df['cluster_id'].fillna(0).apply(lambda x: int(x))
        cluster_id_list = table_df['cluster_id']
        
        cluster_id_list = list(set(cluster_id_list))
        cluster_id_overview_list.append(cluster_id_list)
        
        count += 1
        bar.update(count)
        
overview_df['cluster_id'] = cluster_id_overview_list
overview_df

100% (4355 of 4355) |####################| Elapsed Time: 0:02:56 Time:  0:02:56


,table_id,cluster_id
0,Product_lojapeterpaiva.com.br_September2020.json.gz,"[1738768, 0, 2434654]"
1,Product_sixtysixtysounds.com_September2020.json.gz,"[0, 730377, 894987, 1738768, 2767770, 33570, 936098, 1467318, 1257145, 733504, 836946, 1617106, ..."
2,Product_boticas23.com_September2020.json.gz,"[708960, 0, 12326, 472074, 101291, 1343372, 607246, 1937007, 941488, 434897, 1592018, 33370, 256..."
3,Product_maidstonetreefarm.com_September2020.json.gz,"[0, 1543882, 1187164, 1063839]"
4,Product_ahorroimprimiendo.com_September2020.json.gz,"[0, 38085]"
...,...,...
4350,Product_aktivwholesale.com_September2020.json.gz,"[0, 65383188]"
4351,Product_malinandgoetz.com_September2020.json.gz,"[0, 1257145, 1617106]"
4352,Product_specialisteduski.com_September2020.json.gz,"[0, 251884, 863679]"
4353,Product_thecraftist.com_September2020.json.gz,"[0, 65383188]"


In [64]:
overview_df.to_json(os.path.join(train_test_input_unfiltered_path, 'overview/train_test_overview.json.gz'), compression='gzip', orient='records', lines=True)

In [109]:
overview_detailed = overview_df.explode('cluster_id')
overview_detailed['cluster_id'] = overview_detailed['cluster_id'].apply(lambda x: str(x))
overview_detailed

,table_id,cluster_id
0,Product_lojapeterpaiva.com.br_September2020.json.gz,1738768
0,Product_lojapeterpaiva.com.br_September2020.json.gz,0
0,Product_lojapeterpaiva.com.br_September2020.json.gz,2434654
1,Product_sixtysixtysounds.com_September2020.json.gz,0
1,Product_sixtysixtysounds.com_September2020.json.gz,730377
...,...,...
4352,Product_specialisteduski.com_September2020.json.gz,863679
4353,Product_thecraftist.com_September2020.json.gz,0
4353,Product_thecraftist.com_September2020.json.gz,65383188
4354,Product_area399.com_September2020.json.gz,0


In [91]:
# add row information
from collections import Counter
count = 0
with progressbar.ProgressBar(max_value=len(all_unfiltered_tables)) as bar:
    for table in all_unfiltered_tables:
        table_df = pd.read_json(os.path.join(train_test_input_unfiltered_path,'{}'.format(table)), compression='gzip', 
                                lines=True)
        table_df['cluster_id'] = table_df['cluster_id'].fillna(0).apply(lambda x: int(x))
        cluster_id_list = table_df['cluster_id'].apply(lambda x: str(x))
        cluster_dict = dict(Counter(cluster_id_list))
        cluster_df = pd.DataFrame.from_dict(cluster_dict, orient='index')
        cluster_df['table_id'] = table
        cluster_df['cluster_id'] = cluster_df.index
        cluster_df['cluster_id'] = cluster_df['cluster_id'].apply(lambda x: int(x))
        cluster_df.rename(columns={0: 'num_rows'}, inplace=True)
        cluster_df = cluster_df.reset_index().drop(columns='index')
        
        if count == 0:
            overview_detailed_concat = cluster_df
        if count > 0:
            overview_detailed_concat = pd.concat([table_df_2, cluster_df])
        table_df_2 = overview_detailed_concat
        count += 1
        bar.update(count)
        
overview_detailed_concat = overview_detailed_concat.reindex(columns=['table_id', 'cluster_id', 'num_rows'])
overview_detailed_concat

100% (4355 of 4355) |####################| Elapsed Time: 0:02:22 Time:  0:02:22


,table_id,cluster_id,num_rows
0,Product_lojapeterpaiva.com.br_September2020.json.gz,2434654,1
1,Product_lojapeterpaiva.com.br_September2020.json.gz,1738768,1
2,Product_lojapeterpaiva.com.br_September2020.json.gz,0,1146
0,Product_sixtysixtysounds.com_September2020.json.gz,148199,1
1,Product_sixtysixtysounds.com_September2020.json.gz,36985,1
...,...,...,...
2,Product_specialisteduski.com_September2020.json.gz,0,741
0,Product_thecraftist.com_September2020.json.gz,65383188,1
1,Product_thecraftist.com_September2020.json.gz,0,51
0,Product_area399.com_September2020.json.gz,541658,1


In [94]:
overview_detailed_concat = overview_detailed_concat.drop_duplicates()
overview_detailed_concat.to_json(os.path.join(train_test_input_unfiltered_path, 'overview/train_test_detailed_overview.json.gz'), compression='gzip', orient='records', lines=True)
overview_detailed_concat

,table_id,cluster_id,num_rows
0,Product_lojapeterpaiva.com.br_September2020.json.gz,2434654,1
1,Product_lojapeterpaiva.com.br_September2020.json.gz,1738768,1
2,Product_lojapeterpaiva.com.br_September2020.json.gz,0,1146
0,Product_sixtysixtysounds.com_September2020.json.gz,148199,1
1,Product_sixtysixtysounds.com_September2020.json.gz,36985,1
...,...,...,...
2,Product_specialisteduski.com_September2020.json.gz,0,741
0,Product_thecraftist.com_September2020.json.gz,65383188,1
1,Product_thecraftist.com_September2020.json.gz,0,51
0,Product_area399.com_September2020.json.gz,541658,1


In [122]:
full_table = pd.concat([overview_detailed,pd.get_dummies(overview_detailed['cluster_id'])], axis=1)
full_table = full_table.groupby('table_id').sum().reset_index()
full_table.to_json(os.path.join(train_test_input_unfiltered_path, 'overview/train_test_full_detailed_overview.json.gz'), compression='gzip', orient='records', lines=True)
full_table

,table_id,0,1001195,1001446,1001804,1004137,1004942,1005500,1006110,1007517,...,982918,983488,984421,985,988749,989035,99153,993500,998602,999135
0,Product_12volttechnology.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Product_1clicksafety.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Product_1fodiscount.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Product_1foteam.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Product_1servicecar.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4350,Product_zpcinformatica.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4351,Product_zsmartbuy.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4352,Product_zsyst.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4353,Product_zvejotribuna.net_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Create train test split for large dataset

In [120]:
# split tables into train and test set
all_tables = [file for file in os.listdir(train_test_input_unfiltered_path) if file.endswith('.json.gz')]
train_test_output_path = os.path.join(product_path, 'train_test_split/output_unfiltered_tables')

unique_clusters_df = pd.read_excel(os.path.join(results_path, 'Final_lower threshold.xlsx'))
unique_clusters_list = list(set(unique_clusters_df['cluster_id'].apply(lambda x: str(x)).to_list()))
unique_clusters_list.append(str(0)) # add 0 for the products which are not responding to a cluster_id

X = np.array(full_table['table_id'])
y = np.array(full_table[unique_clusters_list])
msss = MultilabelStratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=42)
for train_index, test_index in msss.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

#print(test_index)
train_set = set(X_train)
print('train set: {}'.format(train_set))
test_set = set(X_test)
print('test set: {}'.format(test_set))

count = 0
with progressbar.ProgressBar(max_value=len(all_tables)) as bar:
    for table in all_tables:
        if table in train_set:
            df = pd.read_json(os.path.join(train_test_input_unfiltered_path, table), compression='gzip', lines=True)
            df.to_json(os.path.join(train_test_output_path, 'large/train', table), compression='gzip', orient='records', lines=True)
        if table in test_set:
            df = pd.read_json(os.path.join(train_test_input_unfiltered_path, table), compression='gzip', lines=True)
            df.to_json(os.path.join(train_test_output_path, 'large/test', table), compression='gzip', orient='records', lines=True)
       
        count += 1
        bar.update(count)

TRAIN: [   0    1    2 ... 4351 4353 4354] TEST: [   5    9   10   13   15   19   20   21   30   34   42   43   53   54
   56   62   64   69   74   77   79   82   87   98  106  111  114  118
  133  138  146  153  164  177  178  179  185  189  195  197  205  216
  221  223  225  235  245  249  256  261  262  269  276  278  281  301
  306  312  321  323  329  336  337  340  341  344  346  348  349  350
  353  356  363  376  377  379  380  382  389  391  394  403  406  409
  413  417  418  427  428  435  443  455  461  462  466  470  475  480
  485  506  507  513  514  517  523  524  529  540  544  553  556  564
  574  582  583  586  593  600  610  614  622  623  624  634  638  645
  656  675  679  680  683  689  699  716  724  735  741  743  750  758
  760  767  770  773  774  784  786  792  793  796  804  805  813  815
  816  824  830  838  846  847  850  851  854  858  862  863  870  872
  875  882  885  887  888  889  890  894  911  916  920  924  930  933
  939  949  951  956  975  9

  0% (0 of 4355) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

TRAIN: [   0    1    2 ... 4349 4351 4352] TEST: [   8   10   11   20   23   25   31   48   49   53   54   59   61   64
   68   73   74   75   80   81   83   89  100  106  110  122  133  136
  137  146  159  166  169  173  175  178  182  188  195  196  200  201
  205  209  211  215  216  223  236  241  246  247  256  258  260  264
  265  267  270  278  280  281  289  292  296  301  307  308  336  341
  342  346  351  352  358  361  369  376  377  381  392  394  396  397
  400  404  412  414  418  419  420  424  428  432  442  445  458  460
  462  465  467  471  472  473  477  480  497  499  501  503  508  509
  514  517  527  533  540  541  546  549  553  563  564  565  581  582
  583  586  589  591  600  602  604  609  610  623  631  635  651  655
  656  658  660  669  673  680  682  689  690  695  697  700  703  704
  708  710  712  716  717  718  720  723  725  731  733  743  748  749
  776  781  782  793  796  803  811  832  850  854  863  868  871  873
  892  896  901  902  905  9

100% (4355 of 4355) |####################| Elapsed Time: 0:06:13 Time:  0:06:13


In [127]:
# check whether distribution worked
test_cluster_id_list = ['0', '1001195', '1001446', '1001804', '985', '988749', '989035', '99153']

for test_cluster_id in test_cluster_id_list:
    table_with_cluster = full_table[full_table[test_cluster_id]==1]
    table_list = list(set(table_with_cluster['table_id']))

    large_train_set = 0
    large_test_set = 0

    for table in table_list:
        if os.path.isfile(os.path.join(train_test_output_path, 'large/train', table)):
            large_train_set += 1
        elif os.path.isfile(os.path.join(train_test_output_path, 'large/test', table)):
            large_test_set += 1

    print('ratio for cluster id {} is: train: {} to test: {}; test ratio: {}'.format(test_cluster_id, large_train_set, 
            large_test_set, large_test_set / (large_train_set + large_test_set)))

ratio for cluster id 0 is: train: 3451 to test: 904; test ratio: 0.20757749712973594
ratio for cluster id 1001195 is: train: 42 to test: 10; test ratio: 0.19230769230769232
ratio for cluster id 1001446 is: train: 12 to test: 5; test ratio: 0.29411764705882354
ratio for cluster id 1001804 is: train: 7 to test: 3; test ratio: 0.3
ratio for cluster id 985 is: train: 8 to test: 3; test ratio: 0.2727272727272727
ratio for cluster id 988749 is: train: 131 to test: 40; test ratio: 0.23391812865497075
ratio for cluster id 989035 is: train: 8 to test: 2; test ratio: 0.2
ratio for cluster id 99153 is: train: 22 to test: 5; test ratio: 0.18518518518518517


## Create train test split for medium dataset

In [129]:
# create overview table for large train set
all_tables_large_train = [file for file in os.listdir(os.path.join(train_test_output_path, 'large/train')) if file.endswith('.json.gz')]
large_train = full_table[full_table['table_id'].isin(all_tables_large_train)]
large_train.to_csv(os.path.join(train_test_output_path, 'large', 'train_set_overview_large.csv'))
large_train

,table_id,0,1001195,1001446,1001804,1004137,1004942,1005500,1006110,1007517,...,982918,983488,984421,985,988749,989035,99153,993500,998602,999135
0,Product_12volttechnology.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Product_1clicksafety.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Product_1fodiscount.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Product_1foteam.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Product_1servicecar.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4347,Product_zorinstechnologies.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4348,Product_zoro.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4349,Product_zotim.com.au_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4351,Product_zsmartbuy.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [130]:
# create overview table for large test set
all_tables_large_test = [file for file in os.listdir(os.path.join(train_test_output_path, 'large/test')) if file.endswith('.json.gz')]
large_test = full_table[full_table['table_id'].isin(all_tables_large_test)]
large_test.to_csv(os.path.join(train_test_output_path, 'large', 'test_set_overview_large.csv'))
large_test

,table_id,0,1001195,1001446,1001804,1004137,1004942,1005500,1006110,1007517,...,982918,983488,984421,985,988749,989035,99153,993500,998602,999135
8,Product_247inktoner.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,Product_2techshop.com_September2020.json.gz,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,Product_321eshop.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,Product_4clik.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23,Product_4run3.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4339,Product_zeuscomics.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4346,Product_zoominformatica.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4350,Product_zpcinformatica.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4353,Product_zvejotribuna.net_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [131]:
# split tables into train and test set
unique_clusters_df = pd.read_excel(os.path.join(results_path, 'Final_lower threshold.xlsx'))
unique_clusters_list = list(set(unique_clusters_df['cluster_id'].apply(lambda x: str(x)).to_list()))
unique_clusters_list.append(str(0)) # add 0 for the products which are not responding to a cluster_id

X = np.array(large_train['table_id'])
y = np.array(large_train[unique_clusters_list])
msss = MultilabelStratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=42)
for train_index, test_index in msss.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

#print(test_index)
train_set = set(X_train)
print('train set: {}'.format(train_set))
test_set = set(X_test)
print('test set: {}'.format(test_set))

count = 0
with progressbar.ProgressBar(max_value=len(all_tables_large_train)) as bar:
    for table in all_tables_large_train:
        if table in train_set:
            df = pd.read_json(os.path.join(train_test_input_unfiltered_path, table), compression='gzip', lines=True)
            df.to_json(os.path.join(train_test_output_path, 'medium/train', table), compression='gzip', orient='records', lines=True)
        if table in test_set:
            df = pd.read_json(os.path.join(train_test_input_unfiltered_path, table), compression='gzip', lines=True)
            df.to_json(os.path.join(train_test_output_path, 'medium/test', table), compression='gzip', orient='records', lines=True)
       
        count += 1
        bar.update(count)

TRAIN: [   0    1    3 ... 3447 3448 3449] TEST: [   2    6    7    9   16   18   19   25   36   38   47   48   49   53
   55   56   61   68   72   74   76   78   88   89  114  115  118  119
  122  126  130  131  136  145  152  160  161  164  165  171  176  179
  180  181  185  186  189  190  197  202  207  209  219  221  222  223
  225  229  232  260  264  267  268  273  275  277  286  287  290  301
  302  305  307  317  324  329  330  347  354  373  374  376  388  390
  393  394  399  410  426  427  431  432  434  439  443  446  450  454
  459  463  466  470  473  475  481  484  486  489  502  504  506  510
  511  514  526  531  536  539  540  541  543  544  548  574  576  577
  582  588  594  597  600  602  614  615  623  626  627  628  629  630
  631  633  636  639  652  659  667  671  674  692  693  696  701  710
  719  722  724  730  731  735  754  761  771  778  783  789  791  794
  803  814  818  831  833  835  837  839  846  851  865  872  875  879
  882  884  888  895  910  9

  0% (4 of 3451) |                       | Elapsed Time: 0:00:00 ETA:   0:01:36

TRAIN: [   0    1    2 ... 3447 3448 3449] TEST: [   7   10   12   13   23   36   37   48   55   61   68   72   80   82
   93   94  103  114  115  122  136  141  144  150  157  161  165  171
  182  183  184  185  188  189  190  197  200  202  204  210  212  214
  218  220  221  227  235  237  240  241  242  244  252  257  258  261
  267  271  273  277  279  283  284  292  294  315  316  328  330  334
  335  337  339  352  354  358  361  362  366  367  372  373  374  375
  381  382  384  386  390  393  420  425  439  446  450  458  460  470
  472  478  494  497  500  502  509  514  517  518  526  540  541  543
  544  548  551  552  559  563  570  572  577  578  580  583  588  593
  598  599  603  604  615  618  622  639  643  645  648  656  684  687
  694  701  705  710  711  727  728  741  743  751  755  757  760  772
  775  778  779  786  788  801  806  810  815  817  822  826  827  831
  834  835  837  842  843  848  849  851  862  871  877  881  882  891
  898  908  909  920  924  9

100% (3451 of 3451) |####################| Elapsed Time: 0:05:03 Time:  0:05:03


In [138]:
# check whether distribution worked
test_cluster_id_list = ['0', '1001195', '1005500', '1006110', '1007517' , '985', '983488', '984421', '999135']

for test_cluster_id in test_cluster_id_list:
    table_with_cluster = large_train[large_train[test_cluster_id]==1]
    table_list = list(set(table_with_cluster['table_id']))

    medium_train_set = 0
    medium_test_set = 0

    for table in table_list:
        if os.path.isfile(os.path.join(train_test_output_path, 'medium/train', table)):
            medium_train_set += 1
        elif os.path.isfile(os.path.join(train_test_output_path, 'medium/test', table)):
            medium_test_set += 1

    print('ratio for cluster id {} is: train: {} to test: {}; test ratio: {}'.format(test_cluster_id, medium_train_set, 
            medium_test_set, medium_test_set / (medium_train_set + medium_test_set)))

ratio for cluster id 0 is: train: 2760 to test: 691; test ratio: 0.2002318168646769
ratio for cluster id 1001195 is: train: 34 to test: 8; test ratio: 0.19047619047619047
ratio for cluster id 1005500 is: train: 5 to test: 1; test ratio: 0.16666666666666666
ratio for cluster id 1006110 is: train: 5 to test: 1; test ratio: 0.16666666666666666
ratio for cluster id 1007517 is: train: 10 to test: 3; test ratio: 0.23076923076923078
ratio for cluster id 985 is: train: 6 to test: 2; test ratio: 0.25
ratio for cluster id 983488 is: train: 5 to test: 2; test ratio: 0.2857142857142857
ratio for cluster id 984421 is: train: 10 to test: 2; test ratio: 0.16666666666666666
ratio for cluster id 999135 is: train: 5 to test: 2; test ratio: 0.2857142857142857


## Create train test split for small dataset 

In [134]:
# create overview table for medium train set
all_tables_medium_train = [file for file in os.listdir(os.path.join(train_test_output_path, 'medium/train')) if file.endswith('.json.gz')]
medium_train = full_table[full_table['table_id'].isin(all_tables_medium_train)]
medium_train.to_csv(os.path.join(train_test_output_path, 'medium', 'train_set_overview_medium.csv'))
medium_train

,table_id,0,1001195,1001446,1001804,1004137,1004942,1005500,1006110,1007517,...,982918,983488,984421,985,988749,989035,99153,993500,998602,999135
0,Product_12volttechnology.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Product_1clicksafety.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Product_1fodiscount.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Product_1foteam.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Product_1servicecar.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4344,Product_zoccoshop.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4347,Product_zorinstechnologies.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4348,Product_zoro.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4349,Product_zotim.com.au_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [136]:
# create overview table for medium test set
all_tables_medium_test = [file for file in os.listdir(os.path.join(train_test_output_path, 'medium/test')) if file.endswith('.json.gz')]
medium_test = full_table[full_table['table_id'].isin(all_tables_medium_test)]
medium_test.to_csv(os.path.join(train_test_output_path, 'medium', 'test_set_overview_medium.csv'))
medium_test

,table_id,0,1001195,1001446,1001804,1004137,1004942,1005500,1006110,1007517,...,982918,983488,984421,985,988749,989035,99153,993500,998602,999135
7,Product_230v.com.ua_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,Product_365rider.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,Product_38thparalleldivers.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16,Product_3dcartstores.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29,Product_7bucket.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4313,Product_younginteriorsflooring.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4317,Product_yourfloorz.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4335,Product_zeiglerchevroletschaumburg.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4345,Product_zonehomeentertainment.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [137]:
# split tables into train and test set
unique_clusters_df = pd.read_excel(os.path.join(results_path, 'Final_lower threshold.xlsx'))
unique_clusters_list = list(set(unique_clusters_df['cluster_id'].apply(lambda x: str(x)).to_list()))
unique_clusters_list.append(str(0)) # add 0 for the products which are not responding to a cluster_id

X = np.array(medium_train['table_id'])
y = np.array(medium_train[unique_clusters_list])
msss = MultilabelStratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=42)
for train_index, test_index in msss.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

#print(test_index)
train_set = set(X_train)
print('train set: {}'.format(train_set))
test_set = set(X_test)
print('test set: {}'.format(test_set))

count = 0
with progressbar.ProgressBar(max_value=len(all_tables_medium_train)) as bar:
    for table in all_tables_medium_train:
        if table in train_set:
            df = pd.read_json(os.path.join(train_test_input_unfiltered_path, table), compression='gzip', lines=True)
            df.to_json(os.path.join(train_test_output_path, 'small/train', table), compression='gzip', orient='records', lines=True)
        if table in test_set:
            df = pd.read_json(os.path.join(train_test_input_unfiltered_path, table), compression='gzip', lines=True)
            df.to_json(os.path.join(train_test_output_path, 'small/test', table), compression='gzip', orient='records', lines=True)
       
        count += 1
        bar.update(count)

TRAIN: [   0    5    6 ... 2757 2758 2759] TEST: [   1    2    3    4   14   16   21   22   25   33   34   35   36   38
   43   45   46   59   62   63   67   69   74   82   86   87   88   91
   92   95   97  100  104  106  110  112  114  117  119  120  133  136
  137  141  147  155  161  170  180  182  197  205  206  225  249  255
  258  260  264  280  285  293  301  305  315  328  337  343  350  357
  358  362  366  369  372  391  395  401  410  418  422  424  433  435
  445  448  450  458  461  463  469  471  473  476  479  487  502  504
  510  517  531  541  545  547  549  557  559  563  574  577  584  587
  601  606  608  616  623  629  633  640  641  645  648  652  658  664
  665  666  675  676  681  682  683  689  691  696  698  702  711  713
  727  728  729  735  757  758  764  768  771  776  788  791  795  812
  813  814  815  818  826  828  830  831  833  834  835  840  842  843
  844  846  853  860  861  863  867  869  876  882  887  888  890  891
  895  896  898  907  919  9

  0% (5 of 2760) |                       | Elapsed Time: 0:00:00 ETA:   0:00:57

TRAIN: [   1    2    3 ... 2757 2758 2759] TEST: [   0    5   12   17   20   30   37   47   50   56   59   61   62   63
   65   68   74   75   78   83   85   87   90   99  102  104  105  107
  109  118  120  127  129  130  131  140  141  145  146  153  156  158
  161  168  172  180  181  191  195  197  199  200  201  204  207  211
  215  222  223  238  240  253  255  256  268  269  272  274  280  287
  297  298  307  308  309  312  313  317  321  326  337  338  339  348
  354  355  359  361  364  368  379  387  403  406  410  413  422  431
  438  443  444  455  458  466  467  471  473  488  492  495  500  513
  514  515  518  519  522  524  525  527  538  546  548  563  566  578
  579  580  582  586  590  593  596  597  601  602  607  621  630  631
  632  636  655  659  675  676  682  686  691  693  703  714  716  723
  726  727  730  736  738  743  747  759  762  766  768  770  773  775
  777  784  788  792  798  803  804  806  810  818  821  830  831  832
  835  842  851  855  859  8

100% (2760 of 2760) |####################| Elapsed Time: 0:03:59 Time:  0:03:59


In [139]:
# check whether distribution worked
test_cluster_id_list = ['0', '1001195', '1005500', '1006110', '1007517' , '985', '983488', '984421', '999135']

for test_cluster_id in test_cluster_id_list:
    table_with_cluster = medium_train[medium_train[test_cluster_id]==1]
    table_list = list(set(table_with_cluster['table_id']))

    small_train_set = 0
    small_test_set = 0

    for table in table_list:
        if os.path.isfile(os.path.join(train_test_output_path, 'small/train', table)):
            small_train_set += 1
        elif os.path.isfile(os.path.join(train_test_output_path, 'small/test', table)):
            small_test_set += 1

    print('ratio for cluster id {} is: train: {} to test: {}; test ratio: {}'.format(test_cluster_id, small_train_set, 
            small_test_set, small_test_set / (small_train_set + small_test_set)))

ratio for cluster id 0 is: train: 2154 to test: 606; test ratio: 0.21956521739130436
ratio for cluster id 1001195 is: train: 27 to test: 7; test ratio: 0.20588235294117646
ratio for cluster id 1005500 is: train: 4 to test: 1; test ratio: 0.2
ratio for cluster id 1006110 is: train: 4 to test: 1; test ratio: 0.2
ratio for cluster id 1007517 is: train: 7 to test: 3; test ratio: 0.3
ratio for cluster id 985 is: train: 2 to test: 4; test ratio: 0.6666666666666666
ratio for cluster id 983488 is: train: 4 to test: 1; test ratio: 0.2
ratio for cluster id 984421 is: train: 8 to test: 2; test ratio: 0.2
ratio for cluster id 999135 is: train: 3 to test: 2; test ratio: 0.4


In [140]:
# create overview table for small train set
all_tables_small_train = [file for file in os.listdir(os.path.join(train_test_output_path, 'small/train')) if file.endswith('.json.gz')]
small_train = full_table[full_table['table_id'].isin(all_tables_small_train)]
small_train.to_csv(os.path.join(train_test_output_path, 'small', 'train_set_overview_small.csv'))
small_train

,table_id,0,1001195,1001446,1001804,1004137,1004942,1005500,1006110,1007517,...,982918,983488,984421,985,988749,989035,99153,993500,998602,999135
1,Product_1clicksafety.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Product_1fodiscount.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Product_1foteam.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Product_1servicecar.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Product_1stdiscountsafety.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4343,Product_zipjosh.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4347,Product_zorinstechnologies.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4348,Product_zoro.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4349,Product_zotim.com.au_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [141]:
# create overview table for small test set
all_tables_small_test = [file for file in os.listdir(os.path.join(train_test_output_path, 'small/test')) if file.endswith('.json.gz')]
small_test = full_table[full_table['table_id'].isin(all_tables_small_test)]
small_test.to_csv(os.path.join(train_test_output_path, 'small', 'test_set_overview_small.csv'))
small_test

,table_id,0,1001195,1001446,1001804,1004137,1004942,1005500,1006110,1007517,...,982918,983488,984421,985,988749,989035,99153,993500,998602,999135
0,Product_12volttechnology.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Product_1stclassflooring.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,Product_43einhalb.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27,Product_6ave.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32,Product_916floors.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4318,Product_yourhomeappliance.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4322,Product_ytsundance.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4326,Product_yuppiechef.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4328,Product_zanes.com_September2020.json.gz,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
